### ch06. 텍스트 분류를 위한 미세 튜닝

In [1]:
from importlib.metadata import version

pkgs = ["matplotlib",
        "numpy",
        "tiktoken",
        "torch",
        "tensorflow",
        "pandas"]

for p in pkgs:
    print(f"{p} 버전: {version(p)}")

matplotlib 버전: 3.10.7
numpy 버전: 1.26.4
tiktoken 버전: 0.11.0
torch 버전: 2.6.0
tensorflow 버전: 2.20.0
pandas 버전: 2.2.3


#### 6.2 데이터셋 준비
- 스팸 및 스팸아님 텍스트 메시지로 구성된 데이터셋을 분류하기 위해 LLM을 미세튜닝합니다.
- 먼저 데이터셋을 다운로드하고 압축을 풉니다.

In [ ]:
# import urllib.request
# import zipfile
# import os
# from pathlib import Path

# url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
# zip_path = "sms_spam_collection.zip"
# extracted_path = "sms_spam_collection"
# data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

# def download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path):
#     if data_file_path.exists():
#         print(f"{data_file_path}가 이미 있어 다운로드 및 압축 해제를 건너 뜁니다.")
#         return
    
#     # 파일을 다운로드 합니다.
#     with urllib.request.urlopen(url) as response:
#         with open(zip_path, "wb") as out_file:
#             out_file.write(response.read())
    
#     # 파일 압축을 풉니다.
#     with zipfile.ZipFile(zip_path, "r") as zip_ref:
#         zip_ref.extractall(extracted_path)
    
#     # .tsv 파일 확장자를 추가합니다.
#     original_file_path = Path(extracted_path) / "SMSSpamCollection"
#     os.rename(original_file_path, data_file_path)
#     print(f"파일이 다운로드되어 {data_file_path}에 저장되었습니다.")
    
# try:
#     download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)
# except (urllib.error.HTTPError, urllib.error.URLError, TimeoutError) as e:
#     print(f"기본 URL 실패: {e}. 백업 URL을 시도합니다...")
#     url = "https://f001.backblazeb2.com/file/LLMs-from-scratch/sms%2Bspam%2Bcollection.zip"
#     download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)

파일이 다운로드되어 sms_spam_collection/SMSSpamCollection.tsv에 저장되었습니다.


- 데이터셋은 탭으로 구분된 텍스트 파일로 저장되며, 판다스 DataFrame으로 로드할 수 있습니다

In [1]:
import pandas as pd
from pathlib import Path

extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"
df = pd.read_csv(data_file_path, sep="\t", header=None, names=["Label", "Text"])
df

,Label,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


- 클래스 분포를 확인해 보면, 데이터에 "스팸"보다 "햄"이 훨신 많다는 것을 알 수 있습니다.

In [5]:
print(df["Label"].value_counts())

Label
ham     4825
spam     747
Name: count, dtype: int64


- 간단하게 하기 위해, 그리고 교육 목적으로 작은 데이터셋을 선호하기 때문에 (LLM을 더 빠르게 미세튜닝할 수 있음), 각 클래스마다
747개의 샘플이 포함되도록 데이터 세트를 서브샘플링(언더샘플링)합니다.

In [6]:
def create_balanced_dataset(df):
    # "스팸" 샘플 개수 세기
    num_spam = df[df["Label"] == "spam"].shape[0]
    
    # "스팸" 샘플 개수와 일치하도록 "햄"을 무작위로 샘플
    ham_subset = df[df["Label"] == "ham"].sample(num_spam, random_state=123)
    
    # "햄"과 "스팸"을 합침
    balanced_df = pd.concat([ham_subset, df[df["Label"]=="spam"]])
    
    return balanced_df

balanced_df = create_balanced_dataset(df)
print(balanced_df["Label"].value_counts())

Label
ham     747
spam    747
Name: count, dtype: int64


- 다음으로 문자열 클래스 레이블 "ham"과 "spam"을 정수 클래스 레이블 0과 1로 변경합니다.

In [7]:
balanced_df["Label"] = balanced_df["Label"].map({"ham": 0, "spam": 1})

In [8]:
balanced_df

,Label,Text
4307,0,Awww dat is sweet! We can think of something t...
4138,0,Just got to &lt;#&gt;
4831,0,"The word ""Checkmate"" in chess comes from the P..."
4461,0,This is wishing you a great day. Moji told me ...
5440,0,Thank you. do you generally date the brothas?
...,...,...
5537,1,Want explicit SEX in 30 secs? Ring 02073162414...
5540,1,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,1,Had your contract mobile 11 Mnths? Latest Moto...
5566,1,REMINDER FROM O2: To get 2.50 pounds free call...


- 데이터셋을 훈련 세트, 검증 세트, 테스트 세트로 무작위 분할하는 함수 정의

In [ ]:
def random_split(df, train_frac, validation_frac):
    # 데이터프레임 전체 섞기
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)
    
    # 분할 인덱스 계산
    train_end = int(len(df) *train_frac)
    validation_end = train_end +  int(len(df) * validation_frac)
    
    # 데이터프레임 분할
    train_df = df[: train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]
    
    return train_df, validation_df, test_df

train_df, validation_df, test_df = random_split(balanced_df, 0.7, 0.1)

train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)
    

### 6.3 데이터로더 만들기
- 텍스트 메시지의 길이가 다르다는 점에 유의. 배치에 여러 훈련 샘플을 결합하려면 다음 중 하나를 수행
    1. 데이터셋 또는 배치에서 가장 짧은 메시지 길이로 모든 메시지를 자르기
    2. 데이터셋 또는 배치에서 가장 긴 메시지 길이로 모든 메시지에 패딩을 추가
- 옵션 2를 선택하고 데이터셋에서 가장 긴 메시지에 맞춰 모든 메시지에 패딩을 추가
- 이를 위해 2장에서 설명한 대로 <|endoftext|>를 패딩 토큰으로 사용

In [2]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


- SpamDataset 클래스는 학습 데이터셋에서 가장 긴 시퀀스를 식별하고 다른 시퀀스에 패딩 토큰을 추가하여 해당 길이에 맟춥니다.

In [11]:
import torch
from torch.utils.data import Dataset

class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        self.data = pd.read_csv(csv_file)
        
        # 텍스트 토큰화
        self.encoded_texts = [
            tokenizer.encode(text) for text in self.data["Text"]
        ]
        
        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length
            # max_length보다 긴 시퀀스 자르기
            self.encoded_texts = [
                encoded_text[:self.max_length] for encoded_text in self.encoded_texts
            ]
            
        # 가장 긴 시퀀스에 맞춰 패딩하기
        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text)) 
            for encoded_text in self.encoded_texts
        ]
        
    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["Label"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )
        
    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        # max_length = 0
        # for encoded_text in self.encoded_texts:
        #     encoded_length = len(encoded_text)
        #     if encoded_length > max_length:
        #         max_length = encoded_length
        # return max_length
        return max(len(encoded_text) for encoded_text in self.encoded_texts)        

In [12]:
train_dataset = SpamDataset(
    csv_file="train.csv",
    max_length=None,
    tokenizer=tokenizer
)

print(train_dataset.max_length)

120


- 또한 검증 세트와 테스트 세트를 가장 긴 훈련 시퀀스에 맞춰 패딩합니다.
- 검증 세트와 테스트 세트 샘플 중 가장 긴 훈련 세트 샘플보다 긴 샘플은 SpamDataset 코드에서
  encoded_text[:set.max_length]를 통해 잘림.
- 이 동작은 완전히 선택 사항이며, 검증 세트와 테스트 의 경우 모두 max_length=None으로 설정해도 잘 작동합니다.

In [14]:
val_dataset = SpamDataset(
    csv_file="validation.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

test_dataset = SpamDataset(
    csv_file="test.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

In [15]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False
)

- 검증 단계로서, 데이터 로더를 반복하고 각 배치에 120개의 토큰으로 구성된 8개의 훈련 샘플이 포함되어 있는지 확인

In [16]:
print("훈련 세트 로더: ")
for input_batch, target_batch in train_loader:
    pass

print("입력 배치 차원: ", input_batch.shape)
print("레이블 배치 차원: ", target_batch.shape)

훈련 세트 로더: 
입력 배치 차원:  torch.Size([8, 120])
레이블 배치 차원:  torch.Size([8])


- 마지막으로, 각 데이터셋의 총 배치 수를 출력해 보겠습니다.

In [17]:
print(f"{len(train_loader)}개 훈련 배치")
print(f"{len(val_loader)}개 검증 배치")
print(f"{len(test_loader)}개 테스트 배치")

130개 훈련 배치
19개 검증 배치
38개 테스트 배치


### 6.4 사전 훈련된 가중치로 모델 초기화하기

In [ ]:
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"

BASE_CONFIG = {
    "vocab_size": 50257,    # 어휘사전 크기
    "context_length": 1024, # 문맥 길이
    "drop_rate": 0.0,       # 드롭아웃 비율
    "qkv_bias": True        # 쿼리-키-값 편향
}